In [1]:
import os
from google.cloud import bigquery as bq
import pandas as pd
import pandas_gbq as pgbq
import matplotlib.pyplot as plt

service_credentials = 'Service_Credentials/Big Query Horse Play-f8b9319b2129.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

In [ ]:
schedules_query = (
"""
SELECT *
FROM `bigquery-public-data.baseball.schedules`
""")

games_query = (
"""
SELECT *
FROM `bigquery-public-data.baseball.games_wide`
WHERE awayTeamName = 'Indians'
OR homeTeamId = 'Indians'
"""
)

baseball_queries = [schedules_query, games_query]
baseball_df = []

In [ ]:
# for query in baseball_queries:
#     df = pgbq.read_gbq(query, dialect='standard')
#     baseball_df.append(df)

In [ ]:
# baseball_df[0].to_hdf('Baseball_Dataset/schedules.h5', key='schedules_df')
# baseball_df[1].to_hdf('Baseball_Dataset/games.h5', key='games_df')

In [4]:
games_df = pd.read_hdf('Baseball_Dataset/games.h5')
schedules_df = pd.read_hdf('Baseball_Dataset/schedules.h5')

In [5]:
schedules_df.head()

,gameId,gameNumber,seasonId,year,type,dayNight,duration,duration_minutes,homeTeamId,homeTeamName,awayTeamId,awayTeamName,startTime,attendance,status,created
0,e14b6493-9e7f-404f-840a-8a680cc364bf,1,565de4be-dc80-4849-a7e1-54bc79156cc8,2016,REG,D,3:07,187,03556285-bdbb-4576-a06d-42f71f46ddc5,Marlins,55714da8-fcaf-4574-8443-59bfb511a524,Cubs,2016-06-26 17:10:00,27318,closed,2016-10-06 06:25:15
1,1f32b347-cbcb-4c31-a145-0e685306d168,1,565de4be-dc80-4849-a7e1-54bc79156cc8,2016,REG,D,3:09,189,03556285-bdbb-4576-a06d-42f71f46ddc5,Marlins,55714da8-fcaf-4574-8443-59bfb511a524,Cubs,2016-06-25 20:10:00,29457,closed,2016-10-06 06:25:15
2,0c2292d1-7398-48be-bf8e-b41dad5e1a43,1,565de4be-dc80-4849-a7e1-54bc79156cc8,2016,REG,D,2:45,165,12079497-e414-450a-8bf2-29f91de646bf,Braves,55714da8-fcaf-4574-8443-59bfb511a524,Cubs,2016-06-11 20:10:00,43114,closed,2016-10-06 06:25:15
3,8fbec734-a15a-42ab-8d51-60790de7750b,1,565de4be-dc80-4849-a7e1-54bc79156cc8,2016,REG,D,3:42,222,12079497-e414-450a-8bf2-29f91de646bf,Braves,55714da8-fcaf-4574-8443-59bfb511a524,Cubs,2016-06-12 17:35:00,31625,closed,2016-10-06 06:25:15
4,89e514d5-fbf5-4b9d-bdac-6ca45bfd18dd,1,565de4be-dc80-4849-a7e1-54bc79156cc8,2016,REG,D,2:44,164,2142e1ba-3b40-445c-b8bb-f1f8b1054220,Phillies,55714da8-fcaf-4574-8443-59bfb511a524,Cubs,2016-06-08 17:05:00,28650,closed,2016-10-06 06:25:15


In [10]:
games_df.venueSurface.unique()

array(['grass', 'turf'], dtype=object)

In [16]:
games_df.pitchTypeDescription.unique()

array(['', 'Fastball', 'Changeup', 'Slider', 'Curveball', 'Cutter',
       'Sinker', 'Splitter', 'Other', 'Intentional Ball', 'Knuckleball',
       'Pitchout', 'Screwball', 'Forkball'], dtype=object)

In [38]:
pitch_types_df = games_df.groupby('pitchTypeDescription')['pitchTypeDescription'].agg('count').sort_values(ascending=False)

In [55]:
def determine_wins(row):
    if row['homeFinalRuns'] > row['awayFinalRuns']:
        value = 1
    else:
        value = 0
    return value

In [63]:
# games_df[['homeFinalRuns', 'awayFinalRuns']].dtype

games_df['HomeWins'] = games_df.apply(determine_wins, axis=1)

In [69]:
games_df[['HomeWins', 'homeFinalRuns', 'awayFinalRuns']]

In [ ]:
games_df[['']]

In [75]:
results_df = games_df[['gameId', 'homeTeamName', 'HomeWins']].groupby('gameId').max()

In [80]:
results_df['HomeFlag'] = 1

In [113]:
games_df.outcomeDescription.unique()

array(['', 'Strike Looking', 'Foul Ball', 'Ball', 'Line Out', 'Dirt Ball',
       'Single', 'Fly Out', 'Strike Swinging', 'Fielders Choice',
       'Ground Out', 'Foul Tip', 'SEE RUNNER(S) OUTCOME', 'Pop Out',
       'Double', 'Single - Out at 2nd', 'Triple', 'Hit by pitch',
       'Strike-swinging-Adv-1st', 'Sacrifice Fly',
       'Reached on error-Adv-2nd', ' Strike Swinging - Out at Home',
       'Homerun', 'Reached on error', 'Balk', 'Intentional Ball',
       'Single-Adv-2nd', 'Sacrifice Bunt', "Fielder's Choice-Adv-2nd",
       'Sacrifice Bunt-Adv-1st', 'Double-Adv-3rd', 'Pitch Out',
       'Double Out at 3rd', 'Single-Adv-3rd', 'Sacrifice Fly-Adv-2nd'],
      dtype=object)

In [114]:
not_bat_mask = ['', 'SEE RUNNER(S) OUTCOME']

In [90]:
homeruns_df = games_df[games_df.outcomeDescription == 'Homerun'][['hitterWeight', 'hitterHeight', 'hitterId', 'hitterLastName', 'hitterFirstName']]

In [109]:
def calculate_bmi(row):
    BMI = row['hitterWeight'] / row['hitterHeight']**2
    return BMI    

In [111]:
homeruns_df['BMI'] = homeruns_df.apply(calculate_bmi, axis=1)

In [120]:
hits_df = games_df[games_df.outcomeDescription.isin(not_bat_mask) == False][['hitterWeight', 'hitterHeight', 'hitterId', 'hitterLastName', 'hitterFirstName', 'outcomeDescription']]

In [122]:
hits_df['IsHomerun'] = hits_df['outcomeDescription'].apply(lambda x: 1 if x == 'Homerun' else 0)

In [126]:
hits_df['BMI'] = hits_df.apply(calculate_bmi, axis=1)

In [127]:
hits_df.head()

,hitterWeight,hitterHeight,hitterId,hitterLastName,hitterFirstName,outcomeDescription,IsHomerun,BMI
8,190,73,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,Romine,Andrew,Strike Looking,0,0.035654
9,200,72,8189ef68-447c-4573-94d5-8d690abbcfbf,Kinsler,Ian,Foul Ball,0,0.038580
10,205,74,2847c4e0-01be-46bd-992e-701ee447e3f5,Upton,Justin,Foul Ball,0,0.037436
11,200,72,8189ef68-447c-4573-94d5-8d690abbcfbf,Kinsler,Ian,Ball,0,0.038580
12,205,74,2847c4e0-01be-46bd-992e-701ee447e3f5,Upton,Justin,Ball,0,0.037436


In [ ]:
hits_df[['']]

In [84]:
results_df[['HomeWins', 'HomeFlag']]

,HomeWins,HomeFlag
gameId,,
023e0a42-7531-4078-916c-650a773a3ce1,0,1
086c6d3f-9279-4d6f-a891-76d398cee052,0,1
091f3226-eae6-4502-abc2-899553e935de,1,1
0b869e49-287b-4970-9247-8c34355f92ea,1,1
0f790e88-2ff6-432e-a53f-8f180d799675,0,1
140596ad-0bdd-49f7-897a-73ff842479b8,0,1
1d9f3e07-3c04-4a27-b669-d042e4959827,1,1
1e0523db-4612-40db-a289-b246764a6d64,1,1
244205b0-b223-4c42-9c01-9f8c0aef1826,1,1


In [70]:
games_df[['homeFinalRuns', 'awayFinalRuns']].corr()

,homeFinalRuns,awayFinalRuns
homeFinalRuns,1.000000,-0.245124
awayFinalRuns,-0.245124,1.000000


In [42]:
pitch_types_df[''] / pitch_types_df.sum()

0.05702907208283552

In [33]:
games_df['out'[games_df.pitchTypeDescription == '']

,gameId,seasonId,seasonType,year,startTime,gameStatus,attendance,dayNight,duration,durationMinutes,...,homeBatter4,homeBatter5,homeBatter6,homeBatter7,homeBatter8,homeBatter9,lineupTeamId,lineupPlayerId,lineupPosition,lineupOrder
0,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,2eab845a-e077-40ba-801e-60cf4506bc31,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,575c19b7-4052-41c2-9f0a-1c5813d02f99,2eab845a-e077-40ba-801e-60cf4506bc31,9,7
1,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,fddd3a55-1d6c-44a0-9dd2-8aaaac172f92,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,80715d0d-0d2a-450f-a970-1b9a3b18c7e7,45273e61-1cac-4596-8658-31c06d773435,1,0
2,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,fddd3a55-1d6c-44a0-9dd2-8aaaac172f92,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,575c19b7-4052-41c2-9f0a-1c5813d02f99,f87ea63b-b73a-49cd-b659-c45c0193c3b8,1,0
3,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,2eab845a-e077-40ba-801e-60cf4506bc31,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,80715d0d-0d2a-450f-a970-1b9a3b18c7e7,e2f70061-dd33-455f-83b7-925f39d7d958,1,0
4,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,fddd3a55-1d6c-44a0-9dd2-8aaaac172f92,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,80715d0d-0d2a-450f-a970-1b9a3b18c7e7,b58ed599-aa4d-407a-a40c-0bb703dd4610,1,0
5,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,fddd3a55-1d6c-44a0-9dd2-8aaaac172f92,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,575c19b7-4052-41c2-9f0a-1c5813d02f99,85f71381-e86e-471e-a344-182ec1f6a76b,1,0
6,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,fddd3a55-1d6c-44a0-9dd2-8aaaac172f92,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,80715d0d-0d2a-450f-a970-1b9a3b18c7e7,f8dc76bf-05b2-4649-a1a0-c65ae2d1a208,7,6
7,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,fddd3a55-1d6c-44a0-9dd2-8aaaac172f92,71580b08-acfc-4d33-9670-6e5a82f389ea,7217e4e7-8fd1-4ed8-8bee-0a5cbb2add1a,80715d0d-0d2a-450f-a970-1b9a3b18c7e7,17e633e5-e7a5-445c-8206-4770254ea2fd,5,5
133,2fea703a-3e98-4b6a-beff-93e550d01c05,565de4be-dc80-4849-a7e1-54bc79156cc8,REG,2016,2016-06-24 23:10:00,closed,37886,N,3:01,181,...,f27a7574-57db-4eeb-8f88-377048806de2,169ed0b1-e34b-4f4b-925b-f3288f24a5e7,2847c4e0-01be-46bd-992e-701ee447e3f5,fddd3a55-1d6c-44a0-9dd2-8aaaac172f92,71580b08-acfc-4d33-9670

In [21]:
games_df.pitchType.sort_values

,gameId,seasonId,seasonType,year,startTime,gameStatus,attendance,dayNight,duration,durationMinutes,...,homeBatter4,homeBatter5,homeBatter6,homeBatter7,homeBatter8,homeBatter9,lineupTeamId,lineupPlayerId,lineupPosition,lineupOrder
